Copyright (c) Microsoft Corporation. All rights reserved.  

Licensed under the MIT License.

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/NotebookVM/how-to-use-azureml/deployment/onnx/onnx-convert-aml-deploy-tinyyolo.png)

# YOLO Real-time Object Detection using ONNX on AzureML

This example shows how to convert the TinyYOLO model from CoreML to ONNX and operationalize it as a web service using Azure Machine Learning services and the ONNX Runtime.

## What is ONNX
ONNX is an open format for representing machine learning and deep learning models. ONNX enables open and interoperable AI by enabling data scientists and developers to use the tools of their choice without worrying about lock-in and flexibility to deploy to a variety of platforms. ONNX is developed and supported by a community of partners including Microsoft, Facebook, and Amazon. For more information, explore the [ONNX website](http://onnx.ai).

## YOLO Details
You Only Look Once (YOLO) is a state-of-the-art, real-time object detection system. For more information about YOLO, please visit the [YOLO website](https://pjreddie.com/darknet/yolo/).

## Prerequisites

To make the best use of your time, make sure you have done the following:

* Understand the [architecture and terms](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture) introduced by Azure Machine Learning
* If you are using an Azure Machine Learning Notebook VM, you are all set. Otherwise, go through the [configuration](../../../configuration.ipynb) notebook to:
    * install the AML SDK
    * create a workspace and its configuration file (config.json)

In [1]:
# Check core SDK version number
import azureml.core

print("SDK version:", azureml.core.VERSION)

SDK version: 1.20.0


#### Install necessary packages

You'll need to run the following commands to use this tutorial:

```sh
pip install onnxmltools
pip install coremltools  # use this on Linux and Mac
pip install git+https://github.com/apple/coremltools  # use this on Windows
```

## Convert model to ONNX

First we download the CoreML model. We use the CoreML model from [Matthijs Hollemans's tutorial](https://github.com/hollance/YOLO-CoreML-MPSNNGraph). This may take a few minutes.

In [2]:
import urllib.request

coreml_model_url = "https://github.com/hollance/YOLO-CoreML-MPSNNGraph/raw/master/TinyYOLO-CoreML/TinyYOLO-CoreML/TinyYOLO.mlmodel"
urllib.request.urlretrieve(coreml_model_url, filename="TinyYOLO.mlmodel")


('TinyYOLO.mlmodel', <http.client.HTTPMessage at 0x7f0f65d9cc50>)

Then we use ONNXMLTools to convert the model.

In [3]:
import onnxmltools
import coremltools

# Load a CoreML model
coreml_model = coremltools.utils.load_spec('TinyYOLO.mlmodel')

# Convert from CoreML into ONNX
onnx_model = onnxmltools.convert_coreml(coreml_model, 'TinyYOLOv2')

# Fix the preprocessor bias in the ImageScaler
for init in onnx_model.graph.initializer:
    if init.name == 'scalerPreprocessor_bias':
        init.dims[1] = 1

# Save ONNX model
onnxmltools.utils.save_model(onnx_model, 'tinyyolov2.onnx')

import os
print(os.path.getsize('tinyyolov2.onnx'))

Using TensorFlow backend.
The maximum opset needed by this model is only 9.


63479035


## Deploying as a web service with Azure ML

### Load Azure ML workspace

We begin by instantiating a workspace object from the existing workspace created earlier in the configuration notebook.

In [4]:
from azureml.core import Workspace

ws = Workspace.from_config()
print(ws.name, ws.location, ws.resource_group, sep = '\n')

mihdemo
eastus
mihhol


### Registering your model with Azure ML

Now we upload the model and register it in the workspace.

In [5]:
from azureml.core.model import Model

model = Model.register(model_path = "tinyyolov2.onnx",
                       model_name = "tinyyolov2",
                       tags = {"onnx": "demo"},
                       description = "TinyYOLO",
                       workspace = ws)

Registering model tinyyolov2


#### Displaying your registered models

You can optionally list out all the models that you have registered in this workspace.

In [6]:
models = ws.models
for name, m in models.items():
    print("Name:", name,"\tVersion:", m.version, "\tDescription:", m.description, m.tags)

Name: best_ridge_model 	Version: 1 	Description: None {}
Name: tinyyolov2 	Version: 2 	Description: TinyYOLO {'onnx': 'demo'}


### Write scoring file

We are now going to deploy our ONNX model on Azure ML using the ONNX Runtime. We begin by writing a score.py file that will be invoked by the web service call. The `init()` function is called once when the container is started so we load the model using the ONNX Runtime into a global session object.

In [7]:
%%writefile score.py
import json
import time
import sys
import os
from azureml.core.model import Model
import numpy as np    # we're going to use numpy to process input and output data
import onnxruntime    # to inference ONNX models, we use the ONNX Runtime

def init():
    global session
    model = Model.get_model_path(model_name = 'tinyyolov2')
    session = onnxruntime.InferenceSession(model)

def preprocess(input_data_json):
    # convert the JSON data into the tensor input
    return np.array(json.loads(input_data_json)['data']).astype('float32')

def postprocess(result):
    return np.array(result).tolist()

def run(input_data_json):
    try:
        start = time.time()   # start timer
        input_data = preprocess(input_data_json)
        input_name = session.get_inputs()[0].name  # get the id of the first input of the model   
        result = session.run([], {input_name: input_data})
        end = time.time()     # stop timer
        return {"result": postprocess(result),
                "time": end - start}
    except Exception as e:
        result = str(e)
        return {"error": result}

Writing score.py


### Setting up inference configuration
First we create a YAML file that specifies which dependencies we would like to see in our container. Please note that you must include azureml-defaults with verion >= 1.0.45 as a pip dependency, because it contains the functionality needed to host the model as a web service.

In [18]:
from azureml.core.conda_dependencies import CondaDependencies 

myenv = CondaDependencies.create(pip_packages=["numpy", "onnxruntime", "azureml-core", "azureml-defaults","pip==20.1.1"])

with open("myenv.yml","w") as f:
    f.write(myenv.serialize_to_string())

Then we create the inference configuration.

In [19]:
from azureml.core.model import InferenceConfig
from azureml.core.environment import Environment


myenv = Environment.from_conda_specification(name="myenv", file_path="myenv.yml")
inference_config = InferenceConfig(entry_script="score.py", environment=myenv)

### Deploy the model

In [20]:
from azureml.core.webservice import AciWebservice

aciconfig = AciWebservice.deploy_configuration(cpu_cores = 1, 
                                               memory_gb = 1, 
                                               tags = {'demo': 'onnx'}, 
                                               description = 'web service for TinyYOLO ONNX model')

The following cell will take a few minutes to run as the model gets packaged up and deployed to ACI.

In [11]:
aci_service_name = 'my-aci-service-tiny-yolo'
print("Service", aci_service_name)
aci_service = Model.deploy(ws, aci_service_name, [model], inference_config, aciconfig)
aci_service.wait_for_deployment(True)
print(aci_service.state)

Service my-aci-service-tiny-yolo
Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running...........................................

In case the deployment fails, you can check the logs. Make sure to delete your aci_service before trying again.

In [12]:
if aci_service.state != 'Healthy':
    # run this command for debugging.
    print(aci_service.get_logs())
    aci_service.delete()

## Success!

If you've made it this far, you've deployed a working web service that does object detection using an ONNX model. You can get the URL for the webservice with the code below.

In [13]:
print(aci_service.scoring_uri)

http://f1d1b211-ab40-4dcd-92e9-c5cb92691ae5.eastus.azurecontainer.io/score


In [21]:
from azureml.core.image import ContainerImage

image_config = ContainerImage.image_configuration(execution_script = "score.py",
                                                  runtime = "python",
                                                  conda_file = "myenv.yml",
                                                  docker_file = "Dockerfile",
                                                  description = "TinyYOLO ONNX Demo",
                                                  tags = {"demoMIH": "onnx"}
                                                 )


image = ContainerImage.create(name = "onnxyolo",
                              models = [model],
                              image_config = image_config,
                              workspace = ws)

image.wait_for_creation(show_output = True)

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/ipykernel_launcher.py:8: DeprecationWarning: ContainerImage class has been deprecated and will be removed in a future release. Please migrate to using Environments. https://docs.microsoft.com/en-us/azure/machine-learning/how-to-use-environments
  
/anaconda/envs/azureml_py36/lib/python3.6/site-packages/ipykernel_launcher.py:15: DeprecationWarning: Image class has been deprecated and will be removed in a future release. Please migrate to using Environments. https://docs.microsoft.com/en-us/azure/machine-learning/how-to-use-environments
  from ipykernel import kernelapp as app


Creating image
Running................................................
Succeeded
Image creation operation finished for image onnxyolo:5, operation "Succeeded"


In [22]:
print(image.image_location)

mihdemo.azurecr.io/onnxyolo:5


In [23]:
from azureml.core.webservice import Webservice
from random import randint

aci_service_name = 'onnx-tinyyolo'+str(randint(0,100))
print("Service", aci_service_name)

aci_service = Webservice.deploy_from_image(deployment_config = aciconfig,
                                           image = image,
                                           name = aci_service_name,
                                           workspace = ws)

aci_service.wait_for_deployment(True)
print(aci_service.state)

Service onnx-tinyyolo99
Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running.......................................................................................
Failed


/anaconda/envs/azureml_py36/lib/python3.6/site-packages/ipykernel_launcher.py:10: DeprecationWarning: deploy_from_image has been deprecated and will be removed in a future release. Please migrate to using Environments. https://docs.microsoft.com/en-us/azure/machine-learning/how-to-use-environments
  # Remove the CWD from sys.path while we load stuff.
Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: a637ea80-ab6b-4694-a47c-f93a33dc1b83
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "statusCode": 400,
  "message": "Aci Deployment failed with exception: Error in entry script, ModuleNotFoundError: No module named 'ruamel', please run print(service.get_logs()) to get details.",
  "details": [
    {
      "code": "CrashLoopBackOff",
      "message": "Error in entry script, ModuleNotFoundError: No module named 'ruamel', please run print(service.get_logs()) to get details."
    }
  ]
}



WebserviceException: WebserviceException:
	Message: Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: a637ea80-ab6b-4694-a47c-f93a33dc1b83
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "statusCode": 400,
  "message": "Aci Deployment failed with exception: Error in entry script, ModuleNotFoundError: No module named 'ruamel', please run print(service.get_logs()) to get details.",
  "details": [
    {
      "code": "CrashLoopBackOff",
      "message": "Error in entry script, ModuleNotFoundError: No module named 'ruamel', please run print(service.get_logs()) to get details."
    }
  ]
}
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Service deployment polling reached non-successful terminal state, current service state: Failed\nOperation ID: a637ea80-ab6b-4694-a47c-f93a33dc1b83\nMore information can be found using '.get_logs()'\nError:\n{\n  \"code\": \"Aci

In [24]:
!az extension add --name azure-iot

!az login

In [28]:
!az account set --subscription "Microsoft Azure Internal Consumption"

In [29]:
# Push the deployment JSON to the IOT Hub
!az iot edge set-modules --device-id edgevm --hub-name a9dmhub --content deployment-iotedgeaml.json

[
  {
    "authentication": {
      "symmetricKey": {
        "primaryKey": "LIZcMzyZtNuKHdVWSEggp9X1bSuQQCpQgD8lkGHNCow=",
        "secondaryKey": "CW3JZ9WZuTYX6a6TdIy56EvLBJgVFI89Ra88FF1Ugt8="
      },
      "type": "sas",
      "x509Thumbprint": {
        "primaryThumbprint": null,
        "secondaryThumbprint": null
      }
    },
    "cloudToDeviceMessageCount": 0,
    "connectionState": "Disconnected",
    "connectionStateUpdatedTime": "0001-01-01T00:00:00+00:00",
    "deviceId": "edgevm",
    "etag": "MzQ2NTg0NzA4",
    "generationId": "637479303933244018",
    "lastActivityTime": "0001-01-01T00:00:00+00:00",
    "managedBy": null,
    "moduleId": "$edgeAgent"
  },
  {
    "authentication": {
      "symmetricKey": {
        "primaryKey": null,
        "secondaryKey": null
      },
      "type": "none",
      "x509Thumbprint": {
        "primaryThumbprint": null,
        "secondaryThumbprint": null
      }
    },
    "cloudToDeviceMessageCount": 0,
    "connectionState": "Disconn

When you are eventually done using the web service, remember to delete it.

In [ ]:
aci_service.delete()